<a href="https://colab.research.google.com/github/kd365/faafall22/blob/main/cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NTSB Cleaning Notebook

## Steps:
1. Import libraries and raw data
2. Initial column drops
3. Cleaning each column


### 1. Import libraries

In [ ]:
from google.colab import files
import pandas as pd
 
uploaded = files.upload()

Saving NTSB_for_cleaning.csv to NTSB_for_cleaning (2).csv


In [ ]:
# setting pandas display options
pd.set_option('display.max_rows', 150)

# reading in NTSB csv file
ntsb_raw = pd.read_csv("NTSB_for_cleaning.csv", encoding='latin-1', low_memory=False)
ntsb_raw = ntsb_raw.drop(['engines_ev_id', 'events_ev_id', 'Unnamed: 0'], axis=1)
# view the dataframe info
ntsb_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58688 entries, 0 to 58687
Data columns (total 86 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   aircraft_ev_id         58688 non-null  object 
 1   far_part               58688 non-null  object 
 2   flight_plan_activated  37159 non-null  object 
 3   damage                 58197 non-null  object 
 4   acft_make              58682 non-null  object 
 5   acft_model             58680 non-null  object 
 6   cert_max_gr_wt         51883 non-null  float64
 7   num_eng                57954 non-null  float64
 8   type_last_insp         55899 non-null  object 
 9   date_last_insp         50765 non-null  object 
 10  afm_hrs_last_insp      28970 non-null  float64
 11  afm_hrs                51868 non-null  float64
 12  type_fly               55905 non-null  object 
 13  dprt_apt_id            55148 non-null  object 
 14  dprt_city              57714 non-null  object 
 15  dp

### 2. Initial Column Drop

In [ ]:
# view data
ntsb_raw.head()

,aircraft_ev_id,far_part,flight_plan_activated,damage,acft_make,acft_model,cert_max_gr_wt,num_eng,type_last_insp,date_last_insp,...,inj_tot_t,wx_cond_basic,Cause_Factor,crew_no,crew_category,crew_age,crew_sex,med_certf,pilot_flying,crew_inj_level
0,20001204X00006,091,NaN,DEST,Beech,300,14100.0,2.0,AAIP,1/14/1999,...,2.0,IMC,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
1,20001204X00008,091,NaN,DEST,Aero Commander,560A,6000.0,2.0,ANNL,10/1/1998,...,4.0,VMC,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
2,20001204X00016,091,NaN,DEST,Beech,95-C55,5300.0,2.0,ANNL,6/22/1998,...,3.0,IMC,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
3,20001204X00017,091,NaN,DEST,Beech,BE-55,5121.0,2.0,ANNL,11/4/1998,...,3.0,IMC,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
4,20001204X00031,091,Y,SUBS,Piper,PA-46-350P,4300.0,1.0,COAW,11/24/1998,...,1.0,IMC,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN


In [17]:
# drop any columns that have more than 40% null values
total_rows = ntsb_raw.shape[0]

def drop_cols(df, df_size):
    # create a list of columns to drop
    drop = []
    # iterate over each column
    for x in df.columns:
        #determine if the ratio of nulls is greater than 30%
        ratio = (df[x].isna().sum()) / df_size
        if ratio > 0.4:
            drop.append(x)
    # drop the columns in the drop list
    df.drop(labels=drop, axis=1)
    print('Cols dropped from df:', drop)
    print(df.shape)
    return df

ntsb_dropped_cols = drop_cols(ntsb_raw, total_rows)      


Cols dropped from df: ['afm_hrs_last_insp', 'dest_same_local', 'phase_flt_spec', 'apt_dir', 'vis_rvr', 'wx_dens_alt', 'wx_int_precip', 'crew_sex']
(58688, 89)


In [18]:
print(ntsb_dropped_cols.shape)

(58688, 89)


### 3. Cleaning Each Column

In [19]:
# function to show all the metrics you will want to see for each column
def col_metrics(df, col='none'):
  print('Unique data: ', df[col].unique())
  print('Value counts: ', df[col].value_counts())
  print('Total nulls: ', df[col].isna().sum())

In [20]:
# column 1: far part
col_metrics(ntsb_dropped_cols, col='far_part')      

# making a copy of the data set as ntsb_1
ntsb_1 = pd.DataFrame.copy(ntsb_dropped_cols)

# remove the K and F from part 091
ntsb_1['far_part'] = ntsb_1['far_part'].str.replace('K', '').str.replace('F', '')
print(ntsb_1['far_part'].unique())

Unique data:  ['091' '135' '137' '091F' 'ARMF' '103' '091K']
Value counts:  091     53660
135      3322
137      1537
091K      120
ARMF       28
091F       11
103        10
Name: far_part, dtype: int64
Total nulls:  0
['091' '135' '137' 'ARM' '103']


In [21]:
# column 2: 'flight_plan_activated'
col_metrics(ntsb_1, 'flight_plan_activated')

# fill na with "U"
ntsb_1['flight_plan_activated'] = ntsb_1['flight_plan_activated'].fillna('U')
print(ntsb_1['flight_plan_activated'].isna().sum())

Unique data:  [nan 'Y' 'N' 'U']
Value counts:  N    26101
Y     9824
U     1234
Name: flight_plan_activated, dtype: int64
Total nulls:  21529
0


In [22]:
# column 3: 'damage'
col_metrics(ntsb_1, 'damage')

# change the one Unk to None
ntsb_1['damage'] = ntsb_1['damage'].str.replace('UNK', 'NONE')

Unique data:  ['DEST' 'SUBS' 'MINR' 'NONE' 'UNK' nan]
Value counts:  SUBS    47583
DEST     8494
MINR     2065
NONE       54
UNK         1
Name: damage, dtype: int64
Total nulls:  491


In [ ]:
# column 3: 'damage'
col_metrics(ntsb_1, 'inj_tot_t')
ev_highest_injury      58688 non-null  object 
 76  inj_tot_t 

In [ ]:
# column 3: 'damage'
col_metrics(ntsb_1, 'ev_highest_injury')

In [23]:
# column 4: 'acft_make'
col_metrics(ntsb_1, 'acft_make')

# there are 6 rosw with nulls, will drop those
ntsb_1 = ntsb_1.dropna(subset=['acft_make'])
print(ntsb_1['acft_make'].isna().sum())

Unique data:  ['Beech' 'Aero Commander' 'Piper' ... 'GONZALEZ MANUEL A' 'BRANDT'
 'HARRIS-RUNYAN']
Value counts:  CESSNA                            15629
PIPER                              9036
BEECH                              4231
Cessna                             1813
Piper                              1089
MOONEY                              821
BELL                                741
ROBINSON HELICOPTER                 591
CIRRUS DESIGN CORP                  578
Beech                               567
BELLANCA                            451
ROBINSON HELICOPTER COMPANY         416
AIR TRACTOR INC                     411
EUROCOPTER                          379
BOEING                              347
AERONCA                             343
MAULE                               330
NORTH AMERICAN                      271
ROBINSON                            258
RAYTHEON AIRCRAFT COMPANY           253
STINSON                             251
CHAMPION                            250
DEHAVI

In [30]:
# converting aircraft make to lower case to avoid duplication due to formatting
ntsb_1['acft_make'] = ntsb_1['acft_make'].str.lower() 
col_metrics(ntsb_1, 'acft_make')


Unique data:  ['beech' 'aero commander' 'piper' ... 'gonzalez manuel a' 'brandt'
 'harris-runyan']
Value counts:  cessna                            17442
piper                             10125
beech                              4798
mooney                              975
bell                                756
robinson helicopter                 595
cirrus design corp                  584
bellanca                            526
robinson helicopter company         430
air tractor inc                     412
eurocopter                          380
aeronca                             377
maule                               372
boeing                              363
robinson                            348
north american                      304
stinson                             288
raytheon aircraft company           285
schweizer                           271
champion                            271
aero commander                      266
luscombe                            257
taylor

In [ ]:
ntsb_1.columns

Index(['aircraft_ev_id', 'far_part', 'flight_plan_activated', 'damage',
       'acft_make', 'acft_model', 'cert_max_gr_wt', 'num_eng',
       'type_last_insp', 'date_last_insp', 'afm_hrs_last_insp', 'afm_hrs',
       'type_fly', 'dprt_apt_id', 'dprt_city', 'dprt_state', 'dprt_time',
       'dest_same_local', 'dest_apt_id', 'dest_city', 'dest_state',
       'phase_flt_spec', 'afm_hrs_since', 'rwy_num', 'rwy_len', 'rwy_width',
       'ifr_equipped_cert', 'eng_no', 'eng_type', 'eng_mfgr', 'hp_or_lbs',
       'carb_fuel_injection', 'eng_time_total', 'eng_time_last_insp',
       'ntsb_no', 'ev_type', 'ev_date', 'ev_dow', 'ev_time', 'ev_city',
       'ev_state', 'ev_year', 'ev_month', 'latitude', 'longitude', 'apt_name',
       'ev_nr_apt_id', 'ev_nr_apt_loc', 'apt_dist', 'apt_dir', 'apt_elev',
       'wx_src_iic', 'wx_obs_time', 'wx_obs_dir', 'wx_obs_fac_id',
       'wx_obs_elev', 'wx_obs_dist', 'light_cond', 'sky_cond_nonceil',
       'sky_nonceil_ht', 'sky_ceil_ht', 'sky_cond_ceil', 'vis_

In [ ]:
ntsb_raw = ntsb_raw.loc[:, ~ntsb_raw.columns.str.contains('^Unnamed')]

In [ ]:
print(ntsb_raw['damage'].unique())

In [ ]:
print(ntsb_raw['type_last_insp'].unique())

In [ ]:
print(ntsb_rv1[['ev_time']].to_string(index=False)) 

In [ ]:
ntsb_rv1['ev_time'] = pd.to_numeric(ntsb_rv1['ev_time'])

In [ ]:
import numpy as np
ntsb_rv1['ev_time'] = ntsb_rv1['ev_time'].astype(np.int64)

In [ ]:
nan_count = ntsb_rv1['ev_time'].isna().sum()
print(nan_count)

In [ ]:
print((ntsb_rv1['ev_time'] == 'NaN').sum())

In [ ]:
#adding leading zeros for military time format
ntsb_rv1['ev_time'] = ntsb_rv1['ev_time'].apply(lambda x: '{0:0>4}'.format(x))
print(ntsb_rv1[['ev_time']].to_string(index=False)) 


In [ ]:
ntsb_rv1['ev_time'] = pd.to_datetime(ntsb_rv1['ev_time'], format = '%H%m')

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
ntsb_rv1['date_last_insp'] =  pd.to_datetime(ntsb_rv1['date_last_insp'], infer_datetime_format=True)


In [ ]:
#ntsb_rv2= ntsb_rv1.drop(['Aircraft_Key'], axis=1)

In [ ]:
print(ntsb_rv1.dtypes)

In [ ]:
ntsb_rv1['wx_obs_time'] =  pd.to_datetime(ntsb_rv1['wx_obs_time'], format = "%H%M")

In [4]:
from google.colab import files
import pandas as pd
 
uploaded = files.upload()

uploaded_ASRS = files.upload()

Saving asrs_1989_2022.csv to asrs_1989_2022.csv


KeyboardInterrupt: ignored

In [8]:
# setting pandas display options
pd.set_option('display.max_rows', 150)

# reading in ASRS csv file
asrs_raw = pd.read_csv("asrs_1989_2022.csv", encoding='latin-1', low_memory=False)
asrs_raw = asrs_raw.drop(['Unnamed: 0'], axis=1)
# view the dataframe info
asrs_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1088 entries, 0 to 1087
Columns: 102 entries, index to LONGITUDE_APT
dtypes: float64(8), int64(4), object(90)
memory usage: 867.1+ KB


In [9]:
# view data
asrs_raw.head()

,index,ACN,Date,Local_Time_of_Day,Local_Code,Local_Code_Type,State,Flight_Conditions,Weather_Elements_Visibility,Light,...,Route_In_Use1_None,IsDeidentified,ICAO,matches,COUNTY_NAME_APT,APT_OWNER_TYPE,ELEV,CNTL_TWR,LATITUDE_APT,LONGITUDE_APT
0,982,664844,198907,1201-1800,HYA,Airport,MA,IMC,3,Daylight,...,NaN,0,HYA,BARNSTABLE MUNI-BOARDMAN/POLANDO FIELD,BARNSTABLE,PU,55.0,Y,41.669336,-70.280356
1,763,428524,199902,1201-1800,IND,Airport,IN,IMC,Icing,Daylight,...,NaN,0,IND,INDIANAPOLIS INTL,MARION,PU,797.0,Y,39.717299,-86.294661
2,782,432022,199902,1801-2400,RYN,Airport,AZ,VMC,NaN,Night,...,NaN,0,RYN,RYAN FIELD,PIMA,PU,2417.0,Y,32.142216,-111.174576
3,770,429700,199902,1801-2400,MCI,Airport,KS,VMC,NaN,Night,...,NaN,0,MCI,KANSAS CITY INTL,PLATTE,PU,1026.0,Y,39.297605,-94.713906
4,761,428224,199902,1201-1800,AWO,Airport,WA,VMC,Icing; Rain; 10,Daylight,...,NaN,0,AWO,ARLINGTON MUNI,SNOHOMISH,PU,142.0,N,48.160750,-122.159028


In [12]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print(asrs_raw.columns.values)

['index' 'ACN' 'Date' 'Local_Time_of_Day' 'Local_Code' 'Local_Code_Type'
 'State' 'Flight_Conditions' 'Weather_Elements_Visibility' 'Light'
 'Ceiling' 'Aircraft_Operator_1' 'Make_Model_Name_1' 'Crew_Size_1'
 'Flight_Plan_1' 'Mission_1' 'Flight_Phase_1' 'Route_In_Use_1'
 'Airspace_1' 'Aircraft_Component' 'Aircraft_Reference' 'Problem'
 'Aircraft_Operator_2' 'Make_Model_Name_2' 'Crew_Size_2' 'Flight_Plan_2'
 'Mission_2' 'Flight_Phase_2' 'Route_In_Use_2' 'Airspace_2'
 'Reporter_Organization_1' 'Function_1' 'Qualification_1' 'Experience_1'
 'ASRS_Num_AccessionNumber_1' 'Reporter_Organization_2' 'Function_2'
 'Qualification_2' 'Experience_2' 'ASRS_Num_AccessionNumber_2'
 'Anomaly_Events' 'Result' 'Contributing_Factors_Situations'
 'Primary_Problem' 'Narrative_1' 'Narrative_2' 'Synopsis'
 'FlightCrewLast90Days' 'FlightCrewTotal' 'FlightCrewType'
 'Aircraft Equip' 'AircraftTest' 'Anomaly_Aircraft Equipment Problem'
 'Anomaly_Conflict' 'Anomaly_Other' 'Anomaly_Ground Excursion'
 'Anomaly_Groun

In [16]:
ntsb_1.columns

NameError: ignored

In [13]:

print(asrs_raw['Problem'].unique())

[nan 'Malfunctioning' 'Improperly Operated' 'Failed'
 'Failed; Malfunctioning' 'Design' 'Improperly Operated; Design'
 'Malfunctioning; Improperly Operated'
 'Improperly Operated; Malfunctioning' 'Design; Malfunctioning']


In [14]:
print(asrs_raw['Anomaly_Events'].unique())

['Deviation / Discrepancy - Procedural FAR; Inflight Event / Encounter Weather / Turbulence; Inflight Event / Encounter VFR In IMC'
 'Aircraft Equipment Problem Less Severe; Deviation - Altitude Overshoot; Deviation / Discrepancy - Procedural Clearance; Inflight Event / Encounter Weather / Turbulence'
 'Aircraft Equipment Problem Critical; Ground Event / Encounter Other / Unknown'
 'Aircraft Equipment Problem Critical; Inflight Event / Encounter Bird / Animal'
 'Aircraft Equipment Problem Critical; Inflight Event / Encounter Weather / Turbulence'
 'Ground Event / Encounter Loss Of Aircraft Control; Ground Excursion Runway; Inflight Event / Encounter Weather / Turbulence'
 'Deviation - Altitude Excursion From Assigned Altitude; Deviation / Discrepancy - Procedural Clearance; Inflight Event / Encounter Weather / Turbulence; Inflight Event / Encounter CFTT / CFIT; Inflight Event / Encounter Loss Of Aircraft Control'
 'Flight Deck / Cabin / Aircraft Event Smoke / Fire / Fumes / Odor; Infli

In [15]:
print(asrs_raw['Result'].unique())

['Flight Crew Diverted; Flight Crew Became Reoriented'
 'Air Traffic Control Issued Advisory / Alert; Flight Crew Overcame Equipment Problem; Flight Crew Returned To Clearance'
 'Aircraft Aircraft Damaged; General Maintenance Action; General None Reported / Taken'
 'Aircraft Aircraft Damaged; Flight Crew Landed in Emergency Condition'
 'Flight Crew Landed in Emergency Condition; General None Reported / Taken'
 'General None Reported / Taken'
 'Air Traffic Control Provided Assistance; Flight Crew Diverted; Flight Crew Regained Aircraft Control; General Declared Emergency'
 'Air Traffic Control Issued New Clearance' nan
 'Aircraft Aircraft Damaged; General None Reported / Taken'
 'Aircraft Aircraft Damaged; Flight Crew Diverted; General Maintenance Action'
 'Air Traffic Control Provided Assistance'
 'Flight Crew Became Reoriented; Flight Crew Took Evasive Action'
 'Air Traffic Control Issued Advisory / Alert; Flight Crew Landed in Emergency Condition; General None Reported / Taken'
 'Fli

In [ ]:
#Walkthrough at https://inria.github.io/scikit-learn-mooc/python_scripts/03_categorical_pipeline_column_transformer.html
#preprocessing different for categorical vs numerical columns

from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(ntsb_raw)
categorical_columns = categorical_columns_selector(ntsb_raw)

In [ ]:
# the columns to be used to create the labels are: inj_tot_t, damage, crew_inj_level, ev_highest_injury, damage